In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/CS594 - DNLP/Assignments/Assignment 1/

/content/drive/My Drive/CS594 - DNLP/Assignments/Assignment 1


In [ ]:
%tensorflow_version 2.x

In [ ]:
import pandas as pd
import numpy as np
import os
# import src

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# pip install autocorrect

     |████████████████████████████████| 1.8MB 3.4MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.1.0-cp36-none-any.whl size=1811979 sha256=d313e7910b127531736fb02bfdea9ce40917a321d357b2cdf80008f9864c1874
  Stored in directory: /root/.cache/pip/wheels/d1/33/69/a42db65d595bb685429c788ab45e11aa2a4b6549298ee38e02
Successfully built autocorrect


In [ ]:
import nltk
# nltk.download()
import string
from nltk.tokenize import word_tokenize
import random
import pickle
from nltk.corpus import stopwords

from autocorrect import Speller
# from pycontractions import Contractions

# from spellchecker import SpellChecker

import re
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer 

from hyperopt import fmin, tpe, hp

In [ ]:
from numpy import asarray
from numpy import zeros
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten, Dropout
from tensorflow.keras.layers import Embedding, SimpleRNN

In [ ]:
def conv_dataframes(pos_path, neg_path):
    with open(pos_path,'r',encoding='latin1') as f:
        data_p = f.readlines()
    print(data_p[11])
    with open(neg_path,'r',encoding='latin1') as f:
        data_n = f.readlines()
    pos_data = shuffle(pd.DataFrame(data_p, columns = ["Doc Text"]))
#     pos_data['Sentiment'] = 1
#     pos_data.columns = ["Doc Text", "Sentiment"]
    neg_data = shuffle(pd.DataFrame(data_n, columns = ["Doc Text"]))
#     neg_data['Sentiment'] = -1
#     neg_data.columns = ["Doc Text", "Sentiment"]
    return pos_data, neg_data


## The code for pos tagging and lemmatize sentence is fron the following link:
 ### https://medium.com/@gaurav5430/using-nltk-for-lemmatizing-sentences-c1bfff963258

In [ ]:
# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [ ]:
def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
#     print(wordnet_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
        # if tag is not None:
        #     lemmatized_sentence.append(lemmatizer.lemmatize(word, tag)) 

    return " ".join(lemmatized_sentence)

In [ ]:
def text_preprocessing(data):
    data["Doc Text"] = data["Doc Text"].str.replace('\n\n', ' ')
    data["Doc Text"] = data["Doc Text"].str.replace('[^\w\s]',' ')
    data["Doc Text"] = data["Doc Text"].str.replace('\s+[a-zA-Z]\s+',' ')
    data["Doc Text"] = data["Doc Text"].str.replace('\^[a-zA-Z]\s+',' ')

    data["Doc Text"] = data["Doc Text"].str.replace('\s+',' ')

    # Remove stop words from text
    data["Doc Text"] = data["Doc Text"].apply(lambda x: " ".join([word for word in x.split() if word not in stop_list]))
    
#     data["Doc Text"] = data["Doc Text"].apply(lambda x: " ".join([lemmatizer.lemmatize(y) for y in x.split()]))
    data["Doc Text"] = data["Doc Text"].apply(lambda x: lemmatize_sentence(x))

    return data

In [ ]:
def create_vocab(pos_data, neg_data):
    temp_pos = text_preprocessing(pos_data)
    temp_neg = text_preprocessing(neg_data)
    temp_pos2 = np.asarray([word_tokenize(re.sub(r"\b[a-zA-Z]\b", " ",i)) for i in temp_pos['Doc Text']])
    temp_neg2 = np.asarray([word_tokenize(re.sub(r"\b[a-zA-Z]\b", " ",i)) for i in temp_neg['Doc Text']]) 
    temp_pos["Doc Text Tokens"] = temp_pos2 
    temp_neg["Doc Text Tokens"] = temp_neg2
    return temp_pos, temp_neg

In [ ]:
file_path  = 'rt-polaritydata' 
pos_path = os.path.join(file_path, 'rt-polarity.pos')
neg_path = os.path.join(file_path, 'rt-polarity.neg')

In [ ]:
pos_data, neg_data = conv_dataframes(pos_path, neg_path)
pos_data.head()

what really surprises about wisegirls is its low-key quality and genuine tenderness . 



,Doc Text
1217,inside the film's conflict-powered plot there ...
624,do we really need a 77-minute film to tell us ...
2548,the film's messages of tolerance and diversity...
2533,a beguiling evocation of the quality that keep...
2461,a glib but bouncy bit of sixties-style slickne...


In [ ]:
# nltk.download('stopwords')

In [ ]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [ ]:
# hm_lines = 5331

# tokenizer = nltk.RegexpTokenizer(r"\w+")
# spell = Speller(lang='en')
# # spell = SpellChecker()

# stop_words = set(stopwords.words('english'))
stop_list = stopwords.words('english')

lemmatizer = WordNetLemmatizer() 

vocab_p, vocab_n = create_vocab(pos_data, neg_data)

vocab_p['Sentiment'] = 1
vocab_n['Sentiment'] = 0

## Splitting the data into training, validation, and testing

In [ ]:
vocab_p_train1, vocab_p_test= train_test_split(vocab_p, test_size = 600)
vocab_p_train, vocab_p_val = train_test_split(vocab_p_train1, test_size = 600)
vocab_p_test.head()

,Doc Text,Doc Text Tokens,Sentiment
4897,unusually dry eye even analytical approach mat...,"[unusually, dry, eye, even, analytical, approa...",1
2930,director writer bille august depicts relations...,"[director, writer, bille, august, depicts, rel...",1
4087,coral reef adventure heavyweight film fight go...,"[coral, reef, adventure, heavyweight, film, fi...",1
3152,grow scooby love movie matthew lillard bear pl...,"[grow, scooby, love, movie, matthew, lillard, ...",1
97,writer director burger imaginatively fan ember...,"[writer, director, burger, imaginatively, fan,...",1


In [ ]:
vocab_n_train1, vocab_n_test= train_test_split(vocab_n, test_size = 600)
vocab_n_train, vocab_n_val = train_test_split(vocab_n_train1, test_size = 600)
vocab_n_test.head()

,Doc Text,Doc Text Tokens,Sentiment
1490,since movie base nicholas sparks best seller k...,"[since, movie, base, nicholas, sparks, best, s...",0
4665,people make fun liking showgirl,"[people, make, fun, liking, showgirl]",0
3940,writer director walter hill co writer david gi...,"[writer, director, walter, hill, co, writer, d...",0
4005,chilly remote emotionally distant piece dull t...,"[chilly, remote, emotionally, distant, piece, ...",0
1515,indifferently implausible popcorn programmer m...,"[indifferently, implausible, popcorn, programm...",0


In [ ]:
training = shuffle(pd.concat([vocab_p_train,vocab_n_train])).reset_index(drop = True)
training['Type'] = 'Training'
validation = pd.concat([vocab_p_val,vocab_n_val]).reset_index(drop = True)
validation['Type'] = 'Validation'
test = pd.concat([vocab_p_test,vocab_n_test]).reset_index(drop = True)
len(validation)

1200

In [ ]:
embeddings = {}
with open("glove.6B.300d.txt", 'r', encoding="utf-8") as file:
    for line in file:
        values = line.split()
        word = values[0]
        vector = asarray(values[1:], dtype='float32')
        embeddings[word] = vector

In [ ]:
def emb_matrix(t,embeddings):
    # creating a embedding matrix for the words in training data, which will be used as weight matrix for embedding layer
    vocab_size = len(t.word_index) + 1    
    embedding_matrix = zeros((vocab_size, 300))
    for word, i in t.word_index.items():
        embedding_vector = embeddings.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix, vocab_size

In [ ]:
def model_rnn(embedding_matrix, vocab_size, max_length, activation, Dropout_rate, kernel_regularizer_coef, activity_regularizer_coef):

    model = Sequential()
    e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable = False)
    model.add(e)
    model.add(SimpleRNN(units = 300, activation = activation))
    model.add(Dropout(Dropout_rate))
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.L1(kernel_regularizer_coef), activity_regularizer=tf.keras.regularizers.L2(activity_regularizer_coef)))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#     print(model.summary())
    return model


In [ ]:
# def objective_func(args):
#     max_length = args['max_length']
# #     af_in_simpleRNN = args['af_in_simpleRNN']
#     batch_size = args['batch_size']
# #     padd = args['padd']
# #     trunc = args['trunc']
# #     kernel_regularizer_coef = args['kernel_regularizer_coef']
# #     activity_regularizer_coef = args['activity_regularizer_coef']
# #     Dropout_rate = args['Dropout_rate']
    
#     encoded_train = tokenise_tf.texts_to_sequences(training['Doc Text'])
#     training_padded = pad_sequences(encoded_train, maxlen=max_length, padding='post', truncating = 'pre')
#     embedding_matrix, vocab_size = emb_matrix(tokenise_tf, embeddings)

#     model = model_rnn(embedding_matrix, vocab_size, max_length, 'tanh', 0.2, 0.001, 0.001)
    
#     model.fit(training_padded, training['Sentiment'], epochs=10, verbose=0, batch_size=batch_size, shuffle =True)    
    
#     encoded_validation = tokenise_tf.texts_to_sequences(validation['Doc Text'])

#     validation_padded = pad_sequences(encoded_validation, maxlen=max_length, padding='post', truncating = 'pre')
#     loss, accuracy = model.evaluate(validation_padded, validation['Sentiment'], verbose=0)
#     return -(accuracy)

# space = {'max_length': hp.choice('max_length',range(4,100)), 
# #          'af_in_simpleRNN': hp.choice('af_in_simpleRNN', ['tanh', 'sigmoid']), 
#         'batch_size': hp.choice('batch_size', [64, 128, 256]),
# #          'padd': hp.choice('padd', ['pre', 'post']),
# #          'trunc': hp.choice('trunc', ['pre', 'post']),
# #          'Dropout_rate': hp.uniform('Dropout_rate', 0, 1),
# #          'kernel_regularizer_coef': hp.uniform('kernel_regularizer_coef', 0, 10),
# #          'activity_regularizer_coef': hp.uniform('activity_regularizer_coef', 0, 10)
#         }
                                
                                
# tokenise_tf = Tokenizer()
# tokenise_tf.fit_on_texts(training['Doc Text'])                                
# best_v_rnn = fmin(objective_func, space, algo=tpe.suggest, max_evals=30)
# print(best_v_rnn)

  0%|          | 0/30 [00:06<?, ?it/s, best loss: ?]


KeyboardInterrupt: ignored

In [ ]:
def objective_func(args):
    max_length = args['max_length']
    af_in_simpleRNN = args['af_in_simpleRNN']
    batch_size = args['batch_size']
    padd = args['padd']
    trunc = args['trunc']
    kernel_regularizer_coef = args['kernel_regularizer_coef']
    activity_regularizer_coef = args['activity_regularizer_coef']
    Dropout_rate = args['Dropout_rate']
    
    encoded_train = tokenise_tf.texts_to_sequences(training['Doc Text'])
    training_padded = pad_sequences(encoded_train, maxlen=max_length, padding=padd, truncating = trunc)
    embedding_matrix, vocab_size = emb_matrix(tokenise_tf, embeddings)

    model = model_rnn(embedding_matrix, vocab_size, max_length, af_in_simpleRNN, Dropout_rate, kernel_regularizer_coef, activity_regularizer_coef)
    
    model.fit(training_padded, training['Sentiment'], epochs=30, verbose=0, batch_size=batch_size, shuffle =True)    
    
    encoded_validation = tokenise_tf.texts_to_sequences(validation['Doc Text'])

    validation_padded = pad_sequences(encoded_validation, maxlen=max_length, padding='post', truncating = 'pre')
    loss, accuracy = model.evaluate(validation_padded, validation['Sentiment'], verbose=0)
    return -(accuracy)

space = {'max_length': hp.choice('max_length',range(4,100)), 
         'af_in_simpleRNN': hp.choice('af_in_simpleRNN', ['tanh', 'sigmoid']), 
        'batch_size': hp.choice('batch_size', [64, 128, 256]),
         'padd': hp.choice('padd', ['pre', 'post']),
         'trunc': hp.choice('trunc', ['pre', 'post']),
         'Dropout_rate': hp.uniform('Dropout_rate', 0, 1),
         'kernel_regularizer_coef': hp.uniform('kernel_regularizer_coef', 0, 1),
         'activity_regularizer_coef': hp.uniform('activity_regularizer_coef', 0, 1)
        }
                                
                                
tokenise_tf = Tokenizer()
tokenise_tf.fit_on_texts(training['Doc Text'])                                
best_v_rnn = fmin(objective_func, space, algo=tpe.suggest, max_evals=40)
print(best_v_rnn)

  2%|▎         | 1/40 [10:41<6:57:17, 641.99s/it, best loss: -0.5]


KeyboardInterrupt: ignored